<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Standard framework
import pandas as pd, numpy as np

# We'll use bqplot for interactive plotting
import bqplot
from bqplot import pyplot as plt

# For our widgets:
import ipywidgets as widgets

In [2]:
# Import the data, the trained model and data preparation pipeline
# See the previous notebook for details

In [3]:
from pycaret.regression import load_model, predict_model

In [4]:
train = pd.read_csv('https://github.com/alu042/DAT158-2022/raw/main/Kaggle/hospital/training_data.csv')
test = pd.read_csv('https://github.com/alu042/DAT158-2022/raw/main/Kaggle/hospital/test_data.csv')
DICT = pd.read_csv('https://github.com/alu042/DAT158-2022/raw/main/Kaggle/hospital/data_dictionary.csv')
METADATA = pd.read_csv('https://github.com/alu042/DAT158-2022/raw/main/Kaggle/hospital/metadata.csv')

In [5]:
# Useful to be able to extract the feature descriptions from DICT:
def remove_whitespace(string):
    return string.replace(" ", "")

DICT['Data fields'] = DICT['Data fields'].apply(remove_whitespace)

def get_feature_descr(feat):
    
    return DICT.loc[DICT['Data fields'] == feat].Descriptions.values[0]

In [6]:
# We add the additional feature discussed in the previous notebook. 
# See the remark in that notebook for the reason why this is not 
# part of the pipeline.
issues = ['dialysisrenalendstage',
 'asthma',
 'irondef',
 'pneum',
 'substancedependence',
 'psychologicaldisordermajor',
 'depress',
 'psychother',
 'fibrosisandother',
 'malnutrition',
 'hemo']

from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='most_frequent')
train[issues] = imp.fit_transform(train[issues])
test[issues] = imp.transform(test[issues])
train.insert(len(train.columns)- 1, 'numberofissues', train[issues].astype('int').sum(axis=1))
test.insert(len(test.columns)- 1, 'numberofissues', test[issues].astype('int').sum(axis=1))

In [7]:
model = load_model('saved_hospital_model_071122', verbose=False)

In [8]:
## Create some widgets for our webapp

In [9]:
## Widgets for entering feature values:

# Date feature

date_widget = widgets.DatePicker(
    value = pd.datetime(2012,3,14)
)


# Categorical features

gender_widget = widgets.Dropdown(
    options = [('Male','M'), ('Female', 'F')],
    value = 'F',
    description = "Gender",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
    
)

facid_widget = widgets.Dropdown(
    options = [('A','A'), ('B', 'B'), ('C', 'C'), ('D', 'D'), ('E', 'E')],
    value = 'A',
    description = "facid",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
    
)

secondarydiagnosisnonicd9_widget = widgets.Dropdown(
    options = [('0',0), ('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', 6), ('7', 7), ('8', 8), ('9', 9), ('10', 10)],
    value = 1,
    description = "secondarydiagnosisnonicd9",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
    
)

In [10]:
# Numerical features


rcount_widget = widgets.Dropdown(
    options = [('0', 0), ('1',1), ('2',2), ('3',3), ('4', 4), ('5 or more', 5)],
    value = 0,
    description = "rcount",
    disabled = False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
    
)


neutrophils_widget = widgets.BoundedFloatText(
    value = 10,
    min=0.,
    max=250.,
    description = 'hematocrit',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)


hematocrit_widget = widgets.BoundedFloatText(
    value = 12,
    min=4.,
    max=25.,
    description = 'hematocrit',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

sodium_widget = widgets.FloatSlider(
    value = 140,
    min=120.,
    max=160.,
    step=0.5,
    description = 'sodium',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

glucose_widget = widgets.BoundedFloatText(
    value = 140,
    min=11.,
    max=270.,
    description = 'glucose',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)


bloodureanitro_widget = widgets.BoundedFloatText(
    value = 14,
    min=1.,
    max=690.,
    description = 'bloodureanitro',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

creatinine_widget = widgets.FloatSlider(
    value = 1,
    min=0.,
    max=3.,
    step=0.1,
    description = 'creatinine',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

bmi_widget = widgets.BoundedFloatText(
    value = 30,
    min=20.,
    max=40.,
    description = 'bmi',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

pulse_widget = widgets.FloatSlider(
    value = 75,
    min=20.,
    max=140.,
    step=1,
    description = 'pulse',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

respiration_widget = widgets.IntSlider(
    value = 7,
    min=0.,
    max=10.,
    step=1,
    description = 'respiration',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='20%')    
)

In [11]:
def get_bool_widget(descr):
    w = widgets.Checkbox(
            value=False,
            description=descr,
            disabled=False,
            style = {'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
            )
    
    return w

In [12]:
# Boolean features:

dialysisrenalendstage_widget = get_bool_widget(get_feature_descr('dialysisrenalendstage'))
asthma_widget = get_bool_widget(get_feature_descr('asthma'))
irondef_widget = get_bool_widget(get_feature_descr('irondef'))
pneum_widget = get_bool_widget(get_feature_descr('pneum'))
substancedependence_widget = get_bool_widget(get_feature_descr('substancedependence'))
psychologicaldisordermajor_widget = get_bool_widget(get_feature_descr('psychologicaldisordermajor'))
depress_widget = get_bool_widget(get_feature_descr('depress'))
psychother_widget = get_bool_widget(get_feature_descr('psychother'))
fibrosisandother_widget = get_bool_widget(get_feature_descr('fibrosisandother'))
malnutrition_widget = get_bool_widget(get_feature_descr('malnutrition'))
hemo_widget = get_bool_widget(get_feature_descr('hemo'))

In [13]:
# Collect all widgets

all_widgets = [
    date_widget,
    gender_widget,
    facid_widget,
    secondarydiagnosisnonicd9_widget,
    rcount_widget,
    hematocrit_widget,
    neutrophils_widget,
    sodium_widget,
    glucose_widget,
    bloodureanitro_widget,
    creatinine_widget,
    bmi_widget,
    pulse_widget,
    respiration_widget,
    dialysisrenalendstage_widget,
    asthma_widget,
    irondef_widget,
    pneum_widget,
    substancedependence_widget,
    psychologicaldisordermajor_widget,
    depress_widget,
    psychother_widget,
    fibrosisandother_widget,
    malnutrition_widget,
    hemo_widget
]

In [14]:
# A function to output the selected feature values
from IPython.core.display import HTML
def selected_feature_values(vdate, rcount, gender, dialysisrenalendstage, asthma,
       irondef, pneum, substancedependence, psychologicaldisordermajor,
       depress, psychother, fibrosisandother, malnutrition, hemo,
       hematocrit, neutrophils, sodium, glucose, bloodureanitro,
       creatinine, bmi, pulse, respiration,
       secondarydiagnosisnonicd9, facid):
    
    html = f""" <h4>Your Selections:</h4>
        <ul>
            <li>Visit date: {vdate}</li>
            <li>rcount: {rcount}</li>
            <li>gender: {gender}</li>
            <li>dialysisrenalendstage: {dialysisrenalendstage}</li>
            <li>asthma: {asthma}</li>
            <li>irondef: {irondef}</li>
            <li>pneum: {pneum}</li>
            <li>substancedependence: {substancedependence}</li>
            <li>psychologicaldisordermajor: {psychologicaldisordermajor}</li>
            <li>depress: {depress}</li>
            <li>psychother: {psychother}</li>
            <li>fibrosisandother: {fibrosisandother}</li>
            <li>malnutrition: {malnutrition}</li> 
            <li>hemo: {hemo}</li> 
            <li>hematocrit: {hematocrit}</li> 
            <li>neutrophils: {neutrophils}</li> 
            <li>sodium: {sodium}</li> 
            <li>glucose: {glucose}</li> 
            <li>bloodureanitro: {bloodureanitro}</li> 
            <li>creatinine: {creatinine}</li> 
            <li>bmi: {bmi}</li> 
            <li>pulse: {pulse}</li> 
            <li>respiration: {respiration}</li> 
            <li>secondarydiagnosisnonicd9: {secondarydiagnosisnonicd9}</li> 
            <li>facid: {facid}</li> 
        </ul>
        <br>
        """
    display(HTML(html))

In [28]:
# A function to output the prediction for the selected feature values
from IPython.core.display import HTML
def get_pred(vdate, rcount, gender, dialysisrenalendstage, asthma,
       irondef, pneum, substancedependence, psychologicaldisordermajor,
       depress, psychother, fibrosisandother, malnutrition, hemo,
       hematocrit, neutrophils, sodium, glucose, bloodureanitro,
       creatinine, bmi, pulse, respiration,
       secondarydiagnosisnonicd9, facid):
    
    
    numberofissues = np.sum([dialysisrenalendstage, asthma,
       irondef, pneum, substancedependence, psychologicaldisordermajor,
       depress, psychother, fibrosisandother, malnutrition, hemo])
    
    id = 76513
    
    pred_df = pd.DataFrame([{
        'id': id,
        'vdate': vdate,
        'gender': gender,
        'facid': facid,
        'secondarydiagnosisnonicd9': secondarydiagnosisnonicd9,
        'rcount': rcount,
        'hematocrit': hematocrit,
        'neutrophils': neutrophils,
        'sodium': sodium,
        'glucose': glucose,
        'bloodureanitro': bloodureanitro,
        'creatinine': creatinine,
        'bmi': bmi,
        'pulse': pulse,
        'respiration': respiration,
        'dialysisrenalendstage': dialysisrenalendstage,
        'asthma': asthma,
        'irondef': irondef,
        'pneum': pneum,
        'substancedependence': substancedependence,
        'psychologicaldisordermajor': psychologicaldisordermajor,
        'depress': depress,
        'psychother': psychother,
        'fibrosisandother': fibrosisandother,
        'malnutrition': malnutrition,
        'hemo': hemo,
        'numberofissues': numberofissues
        
    }])
    

    pred = predict_model(model, data=pred_df)
    
    html = f"""  
      The model predicts a length of stay of {round(pred.Label.values[0],2)} days

    """
    display(HTML(html))

In [16]:
selected = widgets.interactive_output(
    selected_feature_values, 
    {
    'vdate': date_widget,
    'gender': gender_widget,
    'facid': facid_widget,
    'secondarydiagnosisnonicd9': secondarydiagnosisnonicd9_widget,
    'rcount': rcount_widget,
    'hematocrit': hematocrit_widget,
    'neutrophils': neutrophils_widget,
    'sodium': sodium_widget,
    'glucose': glucose_widget,
    'bloodureanitro': bloodureanitro_widget,
    'creatinine': creatinine_widget,
    'bmi': bmi_widget,
    'pulse': pulse_widget,
    'respiration': respiration_widget,
    'dialysisrenalendstage': dialysisrenalendstage_widget,
    'asthma': asthma_widget,
    'irondef': irondef_widget,
    'pneum': pneum_widget,
    'substancedependence': substancedependence_widget,
    'psychologicaldisordermajor': psychologicaldisordermajor_widget,
    'depress': depress_widget,
    'psychother': psychother_widget,
    'fibrosisandother': fibrosisandother_widget,
    'malnutrition': malnutrition_widget,
    'hemo': hemo_widget
    }
)

In [17]:
prediction = widgets.interactive_output(
    get_pred, 
    {
    'vdate': date_widget,
    'gender': gender_widget,
    'facid': facid_widget,
    'secondarydiagnosisnonicd9': secondarydiagnosisnonicd9_widget,
    'rcount': rcount_widget,
    'hematocrit': hematocrit_widget,
    'neutrophils': neutrophils_widget,
    'sodium': sodium_widget,
    'glucose': glucose_widget,
    'bloodureanitro': bloodureanitro_widget,
    'creatinine': creatinine_widget,
    'bmi': bmi_widget,
    'pulse': pulse_widget,
    'respiration': respiration_widget,
    'dialysisrenalendstage': dialysisrenalendstage_widget,
    'asthma': asthma_widget,
    'irondef': irondef_widget,
    'pneum': pneum_widget,
    'substancedependence': substancedependence_widget,
    'psychologicaldisordermajor': psychologicaldisordermajor_widget,
    'depress': depress_widget,
    'psychother': psychother_widget,
    'fibrosisandother': fibrosisandother_widget,
    'malnutrition': malnutrition_widget,
    'hemo': hemo_widget
    }
)

In [18]:
# Deploy the model

In [19]:
title_html = f"""
<h2>Hospital demo app</h2>
Several features are used to predict the length of stay at the hospital. 
"""

feature_values_html = f"""
<hr>
<h3> Enter the feature values here:</h3> 
"""

pred_html = f"""
<hr>
<h3>Model prediction:</h3>
"""

plots_html = f"""
<hr>
<h3> Investigate training data</h3>
You can investigate the values of the features used to train the model here: 
"""

In [20]:
appcont = [widgets.HTML(title_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px')),
          widgets.HTML(feature_values_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px'))]

In [21]:
app = widgets.VBox(appcont)

In [22]:
display(app)

In [23]:
for w in all_widgets:
    display(w)

DatePicker(value=datetime.datetime(2012, 3, 14, 0, 0))

Dropdown(description='Gender', index=1, layout=Layout(width='50%'), options=(('Male', 'M'), ('Female', 'F')), …

Dropdown(description='facid', layout=Layout(width='50%'), options=(('A', 'A'), ('B', 'B'), ('C', 'C'), ('D', '…

Dropdown(description='secondarydiagnosisnonicd9', index=1, layout=Layout(width='50%'), options=(('0', 0), ('1'…

Dropdown(description='rcount', layout=Layout(width='20%'), options=(('0', 0), ('1', 1), ('2', 2), ('3', 3), ('…

...

Checkbox(value=False, description='Flag for depression during encounter', layout=Layout(width='50%'), style=De…

Checkbox(value=False, description='Flag for other psychological disorder during encounter ', layout=Layout(wid…

Checkbox(value=False, description='Flag for fibrosis during encounter', layout=Layout(width='50%'), style=Desc…

Checkbox(value=False, description='Flag for malnutrituion during encounter', layout=Layout(width='50%'), style…

Checkbox(value=False, description='Flag for blood disorder during encounter ', layout=Layout(width='50%'), sty…

In [24]:
#display(selected)

In [25]:
display(widgets.HTML(pred_html, 
                        layout=widgets.Layout(margin='0 0 3em 0', max_width='1000px')))

HTML(value='\n<hr>\n<h3>Model prediction:</h3>\n', layout=Layout(margin='0 0 3em 0', max_width='1000px'))

In [26]:
display(prediction)

Output()

In [27]:
### End-notes ###

# The application we constructed is of course very crude. It is 
# meant as a straightforward example of creating a webapp powered 
# by machine learning using Voilá. 